## Unsupervised Behavioral Phenotyping with 3D Skeletal Pose
Joshua Wu

Timothy Dunn Lab

14 October 2022

Neurodegenerative diseases (like Parkinson's) are characterized by a wide variety of behavioral defects or movement deficits. However, behavior and movement have historically been difficult to quantify and measure.

AIMS (for LID)

<img src='../../CAPTURE_demo/Python_analysis/engine/demo/no_updrs.png' width=40%>

But now with advances in 3D vision - we have DANNCE.

![image](../../CAPTURE_demo/Python_analysis/engine/demo/DANNCE_pipeline.png)

Hopefully, it is intuitive that we need these types of measurements to objectively quantify behavior. But it is unclear how we leverage this data to actually make scientific discoveries.

**GOAL**: Create robust and interpretable un-/self-/semi- supervised analysis frameworks downstream of 3D vision measurements for objectively phenotyping and quantifying behavior.

1. Species-agnostic
2. Batch-effect tolerant
3. Dimension tolerant
4. Statistically validated
5. Interpretable

Previously, there was the CAPTURE MATLAB package (Marshall 2020), which was built off of the MotionMapper (Berman 2014) pipeline for 3D motion capture. It was later adapted for DANNCE.

<img src=https://github.com/jessedmarshall/CAPTURE_demo/raw/master/Common/demo_figure.png width="500">

This notebook represents a faster and more memory efficient Python package for the analysis of behavioral data, which can interface with future frameworks.

In [1]:
import this
from features import *
import DataStruct as ds
import visualization as vis
import interface as itf
import numpy as np
import time
from IPython.display import Video
from embed import Watershed
import copy
from typing import Optional, Union, List
%matplotlib inline

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Load pose predictions and keypoint connectivity information

In [2]:
pstruct = ds.DataStruct(config_path = '../configs/embedding_analysis_ws_r01.yaml')
pstruct.load_connectivity()
pstruct.load_pose()

Detected older version of '.mat' file
Snout
EarR
EarL
SpineF
SpineM
Tail_base_
Forepaw_R
Wrist_R
ForeLimb_R
Forepaw_L
Wrist_L
Forelimb_L
Hindpaw_R
Ankel_R
Hindlimb_R
Hindpaw_L
Ankel_L
Hindlimb_L


Plot some skeletons together

In [3]:
vis.skeleton_vid3D(pstruct.pose_3d,
                   pstruct.connectivity,
                   frames=[1000,500000,2000000],
                   N_FRAMES = 200,
                   dpi = 100,
                   VID_NAME='vid_raw.mp4',
                   SAVE_ROOT=pstruct.out_path)

100%|██████████| 200/200 [00:16<00:00, 11.87it/s]


0

In [4]:
Video(pstruct.out_path + 'vis_vid_raw.mp4', width=600, height=600)

Median filter and align floor across videos

In [5]:
# Separate videos have rotated floor planes - this rotates them back
# Also contains a median filter
pstruct.pose_3d = align_floor(pstruct.pose_3d, 
                              pstruct.exp_ids_full) # median filter size

# Check alignment
vis.skeleton_vid3D(pstruct.pose_3d,
                   pstruct.connectivity,
                   frames=[1000,500000,2000000],
                   N_FRAMES = 200,
                   dpi = 100,
                   VID_NAME='vid_aligned.mp4',
                   SAVE_ROOT=pstruct.out_path)

Fitting and rotating the floor for each video to alignment ... 


100%|██████████| 200/200 [00:17<00:00, 11.70it/s]


0

In [6]:
Video(pstruct.out_path + 'vis_vid_aligned.mp4', width=600, height=600)

Calculating the velocity of the Snout, Mid Spine and Tail Base

In [7]:
# Calculating velocities and standard deviation of velocites over windows
abs_vel, abs_vel_labels  = get_velocities(pstruct.pose_3d, 
                                          pstruct.exp_ids_full, 
                                          pstruct.connectivity.joint_names,
                                          joints=[0,4,5],
                                          widths=[5,11,51])

# Calculate velocities t+10 - t instead of t+1-t
# Should it be an abs val

Calculating absolute velocities ... 


100%|██████████| 7/7 [00:11<00:00,  1.64s/it]


This includes the vector magnitude, x, y, and z velocities as well as the standard deviations across a sliding window of varying length.

In [8]:
print([feat for feat in abs_vel_labels if 'Snout' in feat])

['abs_vel_vec_Snout_5', 'abs_vel_x_Snout_5', 'abs_vel_y_Snout_5', 'abs_vel_z_Snout_5', 'abs_vel_vec_Snout_11', 'abs_vel_x_Snout_11', 'abs_vel_y_Snout_11', 'abs_vel_z_Snout_11', 'abs_vel_vec_Snout_51', 'abs_vel_x_Snout_51', 'abs_vel_y_Snout_51', 'abs_vel_z_Snout_51', 'abs_vel_std_vec_Snout_5', 'abs_vel_std_x_Snout_5', 'abs_vel_std_y_Snout_5', 'abs_vel_std_z_Snout_5', 'abs_vel_std_vec_Snout_11', 'abs_vel_std_x_Snout_11', 'abs_vel_std_y_Snout_11', 'abs_vel_std_z_Snout_11', 'abs_vel_std_vec_Snout_51', 'abs_vel_std_x_Snout_51', 'abs_vel_std_y_Snout_51', 'abs_vel_std_z_Snout_51']


Matching the x-velocity of the Snout with video

In [9]:
snout_vel = abs_vel[:,abs_vel_labels.index('abs_vel_x_Snout_5')]
vis.skeleton_vid3D_features(pstruct.pose_3d,
                            snout_vel,
                            pstruct.connectivity,
                            frames=[500000],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='Snout_abs_vel.mp4',
                            SAVE_ROOT=pstruct.out_path)

100%|██████████| 200/200 [00:21<00:00,  9.23it/s]


0

In [10]:
Video(pstruct.out_path + 'vis_feat_Snout_abs_vel.mp4', width=1200, height=600)

We can probably very easily find a frame with bad tracking by looking at the frame with the highest absolute velocity.

In [13]:
mid_vel = abs_vel[:,abs_vel_labels.index('abs_vel_vec_SpineM_5')]
max_vel_frame = np.argmax(mid_vel)
print("Bad Tracking Frame Number " + str(max_vel_frame))
vis.skeleton_vid3D_features(pstruct.pose_3d,
                            mid_vel,
                            pstruct.connectivity,
                            frames=[max_vel_frame],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='Snout_abs_vel.mp4',
                            SAVE_ROOT=pstruct.out_path)

Bad Tracking Frame Number 531233


100%|██████████| 200/200 [00:22<00:00,  9.00it/s]


0

In [14]:
Video(pstruct.out_path + 'vis_feat_Snout_abs_vel.mp4', width=1200, height=600)

All other features will be egocentric so we will center and lock the front spine onto the x-z axis by rotation.

In [15]:
# Centering all joint locations to mid-spine
pose_locked = center_spine(pstruct.pose_3d)

# Rotates front spine to xz axis
pose_locked = rotate_spine(pose_locked)

vis.skeleton_vid3D(pose_locked,
                   pstruct.connectivity,
                   frames=[50000],
                   N_FRAMES = 200,
                   dpi = 100,
                   VID_NAME='vid_centered.mp4',
                   SAVE_ROOT=pstruct.out_path)

Centering poses to mid spine ...
Rotating spine to xz plane ... 


100%|██████████| 200/200 [00:11<00:00, 16.99it/s]


0

In [16]:
Video(pstruct.out_path + 'vis_vid_centered.mp4', width=600, height=600)

Using this centered and spine-locked pose transformation, we can calculate relative velocities of all keypoints. We leave out the mid spine since it is zeroed.

In [17]:
# Getting relative velocities
rel_vel, rel_vel_labels = get_velocities(pose_locked,
                                         pstruct.exp_ids_full, 
                                         pstruct.connectivity.joint_names,
                                         joints=np.delete(np.arange(18),4),
                                         widths=[5,11,51])

Detected centered pose input - calculating relative velocities ... 


100%|██████████| 7/7 [00:40<00:00,  5.81s/it]


Same as before, this includes the vector magnitude, x, y, and z velocities as well as the standard deviations across a sliding window of varying length.

In [18]:
print([feat for feat in rel_vel_labels if 'Snout' in feat])

['rel_vel_vec_Snout_5', 'rel_vel_x_Snout_5', 'rel_vel_y_Snout_5', 'rel_vel_z_Snout_5', 'rel_vel_vec_Snout_11', 'rel_vel_x_Snout_11', 'rel_vel_y_Snout_11', 'rel_vel_z_Snout_11', 'rel_vel_vec_Snout_51', 'rel_vel_x_Snout_51', 'rel_vel_y_Snout_51', 'rel_vel_z_Snout_51', 'rel_vel_std_vec_Snout_5', 'rel_vel_std_x_Snout_5', 'rel_vel_std_y_Snout_5', 'rel_vel_std_z_Snout_5', 'rel_vel_std_vec_Snout_11', 'rel_vel_std_x_Snout_11', 'rel_vel_std_y_Snout_11', 'rel_vel_std_z_Snout_11', 'rel_vel_std_vec_Snout_51', 'rel_vel_std_x_Snout_51', 'rel_vel_std_y_Snout_51', 'rel_vel_std_z_Snout_51']


Again, let's match relative x-velocity of the snout to a video of the skeleton.

In [20]:
snout_vel = rel_vel[:,rel_vel_labels.index('rel_vel_x_Snout_5')]
vis.skeleton_vid3D_features(pose_locked,
                            snout_vel,
                            pstruct.connectivity,
                            frames=[500000],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='Snout_rel_vel.mp4',
                            SAVE_ROOT=pstruct.out_path)

100%|██████████| 200/200 [00:19<00:00, 10.25it/s]


0

In [21]:
Video(pstruct.out_path + 'vis_feat_Snout_rel_vel.mp4', width=1200, height=600)

Here's the relative z-velocity of the snout during a rearing behavior

In [22]:
snout_vel_rear = rel_vel[:,rel_vel_labels.index('rel_vel_z_Snout_5')]
vis.skeleton_vid3D_features(pose_locked,
                            snout_vel_rear,
                            pstruct.connectivity,
                            frames=[659600],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='Snout_rel_vel_rear.mp4',
                            SAVE_ROOT=pstruct.out_path)

100%|██████████| 200/200 [00:19<00:00, 10.09it/s]


0

In [23]:
Video(pstruct.out_path + 'vis_feat_Snout_rel_vel_rear.mp4', width=1200, height=600)

# Double check why isnt' this signed?

Next, we calculate joint angles.

Hopefully, informative joint angles are preselected in `skeletons.py`.

In [24]:
# Calculating joint angles
angles, angle_labels = get_angles(pose_locked,
                                  pstruct.connectivity.angles)

Calculating joint angles ... 


100%|██████████| 25/25 [00:09<00:00,  2.63it/s]


Each joint angle is represented by 3 numbers corresponding to the projections of that angle onto each of the 3 axis planes (x-y, x-z, y-z).

In [25]:
# 3 Angles for SpineF -> SpineM -> Tail Base
print([label for label in angle_labels if '3_4_5' in label])

['ang_3_4_5_xy', 'ang_3_4_5_xz', 'ang_3_4_5_yz']


Let's go back to that rearing behavior from before look at the xz angle of the back (SpineF -> SpineM -> Tail Base).

In [26]:
back_ang_rear = angles[:,angle_labels.index('ang_3_4_5_xz')]
vis.skeleton_vid3D_features(pose_locked,
                            back_ang_rear,
                            pstruct.connectivity,
                            frames=[659600],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='back_ang_rear.mp4',
                            SAVE_ROOT=pstruct.out_path)

100%|██████████| 200/200 [00:19<00:00, 10.33it/s]


0

In [27]:
Video(pstruct.out_path + 'vis_feat_back_ang_rear.mp4', width=1200, height=600)

Angles calculated are unsigned and from 0 to $\pi$

In [28]:
# Discuss more (circular diff in python)
print(back_ang_rear.max())
print(back_ang_rear.min())

3.141591913615472
5.906200761379809e-07


Let's look at the frames with the largest angles and the lowest angles to make sure there are no non-continuous jumps

In [29]:
vis.skeleton_vid3D_features(pose_locked,
                            back_ang_rear,
                            pstruct.connectivity,
                            frames=[np.argmax(back_ang_rear)],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='back_ang_rear_max.mp4',
                            SAVE_ROOT=pstruct.out_path)

100%|██████████| 200/200 [00:21<00:00,  9.43it/s]


0

In [30]:
Video(pstruct.out_path + 'vis_feat_back_ang_rear_max.mp4', width=1200, height=600)

In [ ]:
vis.skeleton_vid3D_features(pose_locked,
                            back_ang_rear,
                            pstruct.connectivity,
                            frames=[np.argmin(back_ang_rear)],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='back_ang_rear_min.mp4',
                            SAVE_ROOT=pstruct.out_path)

In [ ]:
Video(pstruct.out_path + 'vis_feat_back_ang_rear_min.mp4', width=1200, height=600)

Using these angles we just obtained, we can now calculate angular velocities.

In [ ]:
# Calculating angle velocities
ang_vel, ang_vel_labels = get_angular_vel(angles,
                                          angle_labels,
                                          pstruct.exp_ids_full)

Like we did with positional velocities, we also calculate angular velocities and standard deviations over moving windows of different sizes.

In [ ]:
# 3 Angles for SpineF -> SpineM -> Tail Base
print([label for label in ang_vel_labels if '3_4_5' in label])

Back to that rearing behavior

In [ ]:
back_ang_rear_vel = ang_vel[:,ang_vel_labels.index('avel_3_4_5_xz_5')]
vis.skeleton_vid3D_features(pose_locked,
                            back_ang_rear_vel,
                            pstruct.connectivity,
                            frames=[659600],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='back_ang_rear_vel.mp4',
                            SAVE_ROOT=pstruct.out_path)

In [ ]:
Video(pstruct.out_path + 'vis_feat_back_ang_rear_vel.mp4', width=1200, height=600)

We should also be able to isolate bad tracking with angular velocity

In [ ]:
vis.skeleton_vid3D_features(pstruct.pose_3d,
                            back_ang_rear_vel,
                            pstruct.connectivity,
                            frames=[np.argmax(back_ang_rear_vel)],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='back_ang_rear_vel_max.mp4',
                            SAVE_ROOT=pstruct.out_path)

In [ ]:
Video(pstruct.out_path + 'vis_feat_back_ang_rear_vel_max.mp4', width=1200, height=600)

Finally, we are going to save the egocentric x, y, z coordinates as its own set of features

This code does not calculate anything - it just reshapes the pose and generates labels for each feature.

In [ ]:
# Reshape pose to get egocentric pose features
ego_pose, ego_pose_labels  = get_ego_pose(pose_locked,
                                          pstruct.connectivity.joint_names)

In [ ]:
print([feat for feat in ego_pose_labels if 'Snout' in feat])

Let's just make sure the labels sync up with the features

In [ ]:
snout_z = ego_pose[:,ego_pose_labels.index('ego_euc_Snout_z')]
vis.skeleton_vid3D_features(pose_locked,
                            snout_z,
                            pstruct.connectivity,
                            frames=[659600],
                            N_FRAMES = 200,
                            dpi = 100,
                            VID_NAME='ego_snout_z.mp4',
                            SAVE_ROOT=pstruct.out_path)

In [ ]:
Video(pstruct.out_path + 'vis_feat_ego_snout_z.mp4', width=1200, height=600)

Merge features together and clear some memory

In [ ]:
# Collect all features together
features = np.concatenate([abs_vel, rel_vel, ego_pose, angles, ang_vel], axis=1)
labels = abs_vel_labels + rel_vel_labels + ego_pose_labels + angle_labels + ang_vel_labels

# Clear memory
del pose_locked, abs_vel, rel_vel, ego_pose, angles, ang_vel
del abs_vel_labels, rel_vel_labels, ego_pose_labels, angle_labels, ang_vel_labels

In [ ]:
# Save or read kinematic/wavelet features from h5 file
save_h5(features, labels, path = ''.join([pstruct.out_path,'postural_feats.h5']))
# features, labels =  read_h5(path = ''.join([pstruct.out_path,'postural_feats.h5']))

It's now time for principal component analysis (PCA). PCA is a dimensionality reduction technique which generates orthogonal axes of high variance upon which to project our data. There are many implementations of PCA, but we will use Facebook's Fast Randomized PCA package (`fbpca`), which is significantly faster than most other implementations.

We calculate PCA separately on each feature category to preserve variance and balance the categories. This is in lieu of z-transforming (mean-centering and unit variance) every feature. ** Discussion

In [ ]:
t = time.time()
pc_feats, pc_labels = pca(features,
                          labels,
                          categories = ['abs_vel','rel_vel','ego_euc','ang','avel'],
                          n_pcs = 8,
                          method = 'fbpca')
print("PCA time: " + str(time.time() - t))

del features, labels

Let's see if we can observe clustering among the PCs of any of these feature categories

In [ ]:
def cluster_pcs(pc_feats: np.ndarray,
                pc_labels: List,
                category: str,
                pstruct: ds.DataStruct,
                pcs = np.s_[:2]):
    ''' 
        Makes density plots of the top 2 PCs from a category of features
        Shows the density separately for each condition
    '''
    # Slicing out top PCs from selected category
    cat_pc_idx = [i for i,label in enumerate(pc_labels) if category in label]
    cat_pcs = pc_feats[:, cat_pc_idx[pcs]] # First two pcs

    # Setting up datastruct
    struct = copy.deepcopy(pstruct)
    struct.frame_id = np.arange(0,cat_pcs.shape[0],10)
    struct.embed_vals = cat_pcs[::10]
    struct.exp_id = pstruct.exp_ids_full[::10]
    struct.load_meta()

    # Watershed clustering, also calculates densities
    ws = Watershed(sigma = 15,
                max_clip=1,
                log_out = True,
                pad_factor = 0.05)
    struct.data.loc[:, 'Cluster'] = ws.fit_predict(data = struct.embed_vals)

    # Plot densities
    vis.density_cat(data=struct, column='Condition', watershed=ws, n_col=12, show=True,
                    filepath = ''.join([struct.out_path,'pc_density_',category,'.png']))
    return


Let's see the first two PCs for the absolute velocities

In [ ]:
cluster_pcs(pc_feats, pc_labels,'abs_vel',pstruct)

It's likely that the rapid changes in velocity due to bad tracking frames are affecting the PCA results. Not implemented yet, but will likely need to introduce filtering mechanisms that remove outliers from the PCA calculation.

In [ ]:
cluster_pcs(pc_feats, pc_labels,'ego_euc',pstruct)

The distribution across the first 2 PCs for egocentric euclidean positions appears to be be better, but we're still lacking the ability to resolve the behavioral space. We will need another method - preferably something that is non-linear and a bit more robust ... * *hint hint*

But first!

Although velocities are calculated over rolling windows, the featurization we have so far still lacks the ability to capture complex temporal signals.

To address this, we can leverage the frequency domain through a Morlet wavelet transformation.

Let's see first what a Morlet wavelet looks like.

In [ ]:
from scipy import signal
M = 100
w0 = 3
s = w0*90/(2*np.pi*25)
morlet_wavelet = signal.morlet2(M, s, w0)
plt.plot(morlet_wavelet.imag, label='Imaginary')
plt.plot(morlet_wavelet.real, label='Real')
plt.legend()
plt.show()

Wavelets have the special property of being able to capture local frequency information.

Morlet wavelets have a real component and an imaginary component. When convolving with a time series, you get a complex time series. The magnitude corresponds to the spectral power. The angle from the real axis is the phase.

<img src=https://mathvault.ca/wp-content/uploads/Euler-formula-diagram.png width="500">

Here we calculate the power series for each PC across several frequencies of wavelets

In [ ]:
wlet_feats, wlet_labels = wavelet(pc_feats, 
                                  pc_labels, 
                                  pstruct.exp_ids_full,
                                  sample_freq = 90,
                                  freq = np.linspace(1,25,25),
                                  w0 = 5)

In [ ]:
print(wlet_labels[:25])

Let's plot the spectrogram for the first principal component

In [ ]:
plt.pcolormesh(np.arange(180),np.linspace(1,25,25), wlet_feats[:180,:25].T, cmap='viridis', shading='gouraud')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.colorbar()
plt.show()

In [ ]:
# Save or read kinematic/wavelet features from h5 file
save_h5(wlet_feats, wlet_labels, path = ''.join([pstruct.out_path,'kinematic_feats.h5']))

We now use PCA to reduce the dimensions of the new wavelet features, and consolidate with previous PC scores.

In [ ]:
pc_wlet, pc_wlet_labels = pca(wlet_feats,
                              wlet_labels,
                              categories = ['wlet_abs_vel','wlet_rel_vel','wlet_ego_euc','wlet_ang','wlet_avel'],
                              n_pcs = 8,
                              method = 'fbpca')
del wlet_feats, wlet_labels
pc_feats = np.hstack((pc_feats, pc_wlet))
pc_labels += pc_wlet_labels
del pc_wlet, pc_wlet_labels

In [ ]:
# Save or read PCA-reduced features from h5 file
save_h5(pc_feats, pc_labels, path = ''.join([pstruct.out_path,'pca_feats.h5']))
# pc_feats, pc_labels = read_h5(path = ''.join([pstruct.out_path,'pca_feats.h5']))

Time to run the rest of the analysis

In [ ]:
# Load in analysis params
params = itf.read_params_config(config_path = '../configs/fitsne.yaml')

In [ ]:
# Prepare DataStruct
pstruct.features = pc_feats[::params['downsample']]
pstruct.frame_id = np.arange(0,pc_feats.shape[0],params['downsample'])
pstruct.exp_id = pstruct.exp_ids_full[::params['downsample']]
pstruct.downsample = params['downsample']
pstruct.load_meta()

# Run the rest of the analysis
pstruct = itf.run_analysis(params_config = '../configs/fitsne.yaml',
                 ds = pstruct)

In [ ]:
# import pickle
# pstruct = pickle.load(open('/home/exx/Desktop/GitHub/results/R01_all/fitsne/datastruct.p','rb'))

t-Distributed Stochastic Neighbor Embedding (t-SNE) is a non-linear 2D embedding and visualization algorithm. 

Through a stochastic process, t-SNE works by minimizing the KL divergence between a Gaussian distribution of distances in the high-D space and a Student's t-distribution of distances in the embedding space.

The intuition is that points that are close together in high-D space should remain relatively close in the 2D map. And if clusters of data exist in the high-D space, they should be well represented in the low-D space.

In [ ]:
vis.scatter(pstruct.embed_vals, show=True, 
            filepath=''.join([pstruct.out_path, 'final_scatter.png']))

Each point in this scatter plot represents 1 frame of video. You can see some large clusters as well some smaller clusters. These should represent different behavioral categories.

We can convert the above plot into a density map, and segment out the behavior clusters using a watershed segmentation algorithm.

In [ ]:
vis.density(pstruct.ws.density, pstruct.ws.borders, show=True,
            filepath = ''.join([pstruct.out_path,'final_density.png']))

Each border surrounds what we hope to be distinct behavioral categories. The entire map as a whole represents the behavioral expression of the animals in this dataset.

Let's see how behavioral expression varies across conditions.

In [ ]:
vis.density_cat(data=pstruct, column='Condition', watershed=pstruct.ws, n_col=4, show=True,
                filepath = ''.join([pstruct.out_path,'density_by_condition.png']))

There appears to be a high amount of separation across conditions. Could be very likely that we are experiencing batch effects as some overlap should be expected. Will need further investigation.

Let's look at the density per video.

In [ ]:
vis.density_cat(data=pstruct, column='exp_id', watershed=pstruct.ws, n_col=4, show=True,
                filepath = ''.join([pstruct.out_path,'density_by_video.png']))

Yes - there definitely appear to be some batch effects

We can use `plotly.express` to create interactive maps.

In [ ]:
import plotly.express as px
fig = px.imshow(pstruct.ws.watershed_map)
fig.show()

And then sample and plot skeletons from various clusters of interest.

In [ ]:
vis.skeleton_vid3D_cat(pstruct,
                       'Cluster',
                       labels = [100,96,112,17])

In [ ]:
Video(pstruct.out_path + 'skeleton_vids/vis_Cluster_17.mp4', width=1200, height=600)

By loading back in our features, we can hope to classify clusters using some basic heuristics.

In [ ]:
# Load features back in
features, labels =  read_h5(path = '/home/exx/Desktop/GitHub/results/R01_all/postural_feats.h5')
features = features[::params['downsample'], :]

In [ ]:
# Z-Transformation
features -= features.mean(axis=0)
feat_std = np.std(features,axis=0)
features = features[:,feat_std!=0] # Remove features with no variance
features = features/feat_std[feat_std!=0]
labels = [label for i, label in enumerate(labels) if feat_std[i]!=0]

Let's plot the expression of the head angular velocity across the map

In [ ]:
# print([label for label in labels if 'avel' in label])

# vis.density_feat(pstruct, pstruct.ws, features, labels, 'avel_0_3_4_xy_5')

In [ ]:
key = 'ego_euc_Snout_z'
feat_key = features[:,labels.index(key)]
f = plt.figure()
extent = [*pstruct.ws.hist_range[0], *pstruct.ws.hist_range[1]]
ax = f.add_subplot(111)
# ax.plot(pstruct.ws.borders[:,0],pstruct.ws.borders[:,1],'.k',markersize=0.1)
# ax.scatter(pstruct.ws.borders, zorder=1, extent=extent)
ax.scatter(pstruct.embed_vals[::5,0], pstruct.embed_vals[::5,1],s=0.1,
           c=feat_key[::5], cmap = 'viridis')
ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect('auto')
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
plt.show()

We can also obtain the most differentially expressed features for different clusters

In [ ]:
def get_max_feats_cluster(cluster,
                          features,
                          labels,
                          pstruct):
    feat_cluster = features[pstruct.data['Cluster'] == cluster,:]

    feat_means = feat_cluster.mean(axis=0)
    sorted_idx = np.argsort(feat_means)
    sorted_feats = feat_means[sorted_idx]
    sorted_labels = np.array(labels)[sorted_idx].tolist()

    return sorted_labels[-50:]

In [ ]:
top_feats = get_max_feats_cluster(97,
                                  features,
                                  labels,
                                  pstruct)
print(top_feats)

To be continued....